In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

from sklearn.datasets import load_digits
digits = load_digits()

In [2]:
X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[:1617]
y = digits.target[:1617]

In [3]:
necoc = 100
generations = int(1500/necoc)
generations

15

In [4]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [5]:
def get_winner(class1):
    
    numbers = [*range(10)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1617
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-mnist2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random
while True:
    number_codes = []
    for i in range(2**(10-1), 2**10-1):
        for j in bin(i).split('b')[1]:
            number_codes.append(int(j))
    number_codes = np.array(number_codes).reshape(2**9 - 1,10).T     

    slice = random.sample(range(511), necoc)

    number_codes = number_codes[:,slice].tolist()

    classes = []
    for i in range(necoc):
        code_list = np.array(number_codes).T[i]
        classes.append(np.where(code_list==1)[0].tolist())

    matrix_errors = []

    for i in range(9):
        for j in range(i+1,10):
            matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
    min_error = np.min(matrix_errors)
    if min_error != 0:
        print(min_error)
        #print(number_codes)
        break

37


In [7]:
len(number_codes[0])

100

In [8]:
winner_list = []

for class1 in classes:
    winner = get_winner(class1)
    winner_list.append(winner)

number of class1:646
number of class2:971

 ****** Running generation 0 ****** 

Population's average fitness: 0.51073 stdev: 0.09261
Best fitness: 0.72727 - size: (2, 13) - species 2 - id 151
Average adjusted fitness: 0.204
Mean genetic distance 2.887, standard deviation 0.485
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 21.477 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.55757 stdev: 0.09125
Best fitness: 0.74335 - size: (2, 13) - species 19 - id 356
Average adjusted fitness: 0.253
Mean genetic distance 3.034, standard deviation 0.503
Population of 204 members in 28 species
Total extinctions: 0
Generation time: 20.360 sec (20.919 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.57937 stdev: 0.09106
Best fitness: 0.77365 - size: (3, 14) - species 19 - id 541
Average adjusted fitness: 0.290
Mean genetic distance 3.058, standard deviation 0.486
Population of 202 members in 31 species
Total exti

Population's average fitness: 0.62939 stdev: 0.13548
Best fitness: 0.79468 - size: (6, 22) - species 18 - id 1283
Average adjusted fitness: 0.326
Mean genetic distance 3.153, standard deviation 0.485
Population of 201 members in 32 species
Total extinctions: 0
Generation time: 31.757 sec (26.821 average)

 ****** Running generation 10 ****** 

Population's average fitness: 0.63370 stdev: 0.13475
Best fitness: 0.79468 - size: (6, 22) - species 18 - id 1283
Average adjusted fitness: 0.332
Mean genetic distance 3.158, standard deviation 0.475
Population of 204 members in 32 species
Total extinctions: 0
Generation time: 29.551 sec (27.731 average)

 ****** Running generation 11 ****** 

Population's average fitness: 0.63586 stdev: 0.12923
Best fitness: 0.79468 - size: (6, 22) - species 18 - id 1283
Average adjusted fitness: 0.326
Mean genetic distance 3.151, standard deviation 0.481
Population of 205 members in 33 species
Total extinctions: 0
Generation time: 29.520 sec (28.409 average)

 

Population's average fitness: 0.67800 stdev: 0.19063
Best fitness: 0.84539 - size: (3, 15) - species 4 - id 371
Average adjusted fitness: 0.464
Mean genetic distance 3.056, standard deviation 0.475
Population of 204 members in 32 species
Total extinctions: 0
Generation time: 33.052 sec (24.554 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.67992 stdev: 0.19024
Best fitness: 0.84601 - size: (3, 17) - species 4 - id 823
Average adjusted fitness: 0.470
Mean genetic distance 3.102, standard deviation 0.483
Population of 199 members in 33 species
Total extinctions: 0
Generation time: 36.883 sec (27.019 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.69099 stdev: 0.18834
Best fitness: 0.84601 - size: (3, 17) - species 4 - id 823
Average adjusted fitness: 0.509
Mean genetic distance 3.118, standard deviation 0.472
Population of 197 members in 34 species
Total extinctions: 0
Generation time: 23.744 sec (26.474 average)

 ****** R

Population's average fitness: 0.66750 stdev: 0.09717
Best fitness: 0.82127 - size: (6, 19) - species 2 - id 1858
Average adjusted fitness: 0.278
Mean genetic distance 3.206, standard deviation 0.480
Population of 198 members in 32 species
Total extinctions: 0
Generation time: 22.143 sec (21.299 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.65529 stdev: 0.11256
Best fitness: 0.82127 - size: (6, 19) - species 2 - id 1858
Average adjusted fitness: 0.276
Mean genetic distance 3.201, standard deviation 0.478
Population of 201 members in 32 species
Total extinctions: 0
Generation time: 22.789 sec (21.548 average)

 ****** Running generation 14 ****** 

Population's average fitness: 0.66031 stdev: 0.10596
Best fitness: 0.82127 - size: (6, 19) - species 2 - id 1858
Average adjusted fitness: 0.282
Mean genetic distance 3.194, standard deviation 0.468
Population of 194 members in 32 species
Total extinctions: 0
Generation time: 22.861 sec (21.763 average)
numbe

Population's average fitness: 0.66771 stdev: 0.09093
Best fitness: 0.76747 - size: (5, 19) - species 4 - id 805
Average adjusted fitness: 0.368
Mean genetic distance 3.211, standard deviation 0.465
Population of 203 members in 42 species
Total extinctions: 0
Generation time: 21.326 sec (19.736 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.66487 stdev: 0.09920
Best fitness: 0.76747 - size: (5, 19) - species 4 - id 805
Average adjusted fitness: 0.418
Mean genetic distance 3.220, standard deviation 0.470
Population of 199 members in 44 species
Total extinctions: 0
Generation time: 21.533 sec (19.961 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.67107 stdev: 0.08775
Best fitness: 0.77056 - size: (4, 19) - species 14 - id 1291
Average adjusted fitness: 0.374
Mean genetic distance 3.227, standard deviation 0.480
Population of 197 members in 44 species
Total extinctions: 0
Generation time: 21.566 sec (20.139 average)

 ******

...

Population's average fitness: 0.65671 stdev: 0.12502
Best fitness: 0.79777 - size: (5, 19) - species 4 - id 1538
Average adjusted fitness: 0.345
Mean genetic distance 3.162, standard deviation 0.486
Population of 204 members in 45 species
Total extinctions: 0
Generation time: 19.321 sec (17.689 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.64498 stdev: 0.13239
Best fitness: 0.79777 - size: (5, 19) - species 4 - id 1538
Average adjusted fitness: 0.335
Mean genetic distance 3.184, standard deviation 0.486
Population of 203 members in 46 species
Total extinctions: 0
Generation time: 18.814 sec (17.915 average)

 ****** Running generation 14 ****** 

Population's average fitness: 0.64735 stdev: 0.13942
Best fitness: 0.79777 - size: (5, 19) - species 4 - id 1538
Average adjusted fitness: 0.346
Mean genetic distance 3.167, standard deviation 0.493
Population of 202 members in 46 species
Total extinctions: 0
Generation time: 18.662 sec (18.094 average)
numbe

Population's average fitness: 0.83237 stdev: 0.16868
Best fitness: 0.98083 - size: (5, 17) - species 35 - id 800
Average adjusted fitness: 0.714
Mean genetic distance 3.111, standard deviation 0.513
Population of 204 members in 37 species
Total extinctions: 0
Generation time: 16.966 sec (15.901 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.84676 stdev: 0.13778
Best fitness: 0.99011 - size: (5, 19) - species 35 - id 1182
Average adjusted fitness: 0.763
Mean genetic distance 3.102, standard deviation 0.512
Population of 198 members in 37 species
Total extinctions: 0
Generation time: 17.722 sec (16.128 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.84167 stdev: 0.16223
Best fitness: 0.99011 - size: (5, 19) - species 35 - id 1182
Average adjusted fitness: 0.739
Mean genetic distance 3.100, standard deviation 0.515
Population of 194 members in 38 species
Total extinctions: 0
Generation time: 17.280 sec (16.256 average)

 ***

Population's average fitness: 0.47450 stdev: 0.09250
Best fitness: 0.68275 - size: (2, 13) - species 5 - id 117
Average adjusted fitness: 0.151
Mean genetic distance 2.877, standard deviation 0.437
Population of 200 members in 24 species
Total extinctions: 0
Generation time: 14.294 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.53963 stdev: 0.08994
Best fitness: 0.71614 - size: (3, 15) - species 12 - id 262
Average adjusted fitness: 0.216
Mean genetic distance 2.995, standard deviation 0.464
Population of 200 members in 28 species
Total extinctions: 0
Generation time: 15.172 sec (14.733 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.54382 stdev: 0.08982
Best fitness: 0.71614 - size: (3, 15) - species 12 - id 262
Average adjusted fitness: 0.237
Mean genetic distance 2.959, standard deviation 0.464
Population of 203 members in 28 species
Total extinctions: 0
Generation time: 15.562 sec (15.009 average)

 ****** Running generati

Population's average fitness: 0.58531 stdev: 0.07755
Best fitness: 0.77922 - size: (5, 20) - species 11 - id 1442
Average adjusted fitness: 0.196
Mean genetic distance 3.105, standard deviation 0.487
Population of 202 members in 41 species
Total extinctions: 0
Generation time: 18.178 sec (16.665 average)

 ****** Running generation 10 ****** 

Population's average fitness: 0.58499 stdev: 0.07527
Best fitness: 0.77922 - size: (5, 20) - species 11 - id 1442
Average adjusted fitness: 0.200
Mean genetic distance 3.147, standard deviation 0.486
Population of 211 members in 41 species
Total extinctions: 0
Generation time: 18.493 sec (17.103 average)

 ****** Running generation 11 ****** 

Population's average fitness: 0.59025 stdev: 0.07814
Best fitness: 0.77922 - size: (5, 20) - species 11 - id 1442
Average adjusted fitness: 0.208
Mean genetic distance 3.157, standard deviation 0.481
Population of 200 members in 41 species
Total extinctions: 0
Generation time: 19.749 sec (17.587 average)

 

Population's average fitness: 0.55917 stdev: 0.06588
Best fitness: 0.68213 - size: (4, 16) - species 18 - id 664
Average adjusted fitness: 0.196
Mean genetic distance 3.144, standard deviation 0.475
Population of 201 members in 36 species
Total extinctions: 0
Generation time: 16.294 sec (15.369 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.56297 stdev: 0.06985
Best fitness: 0.70377 - size: (4, 19) - species 22 - id 811
Average adjusted fitness: 0.206
Mean genetic distance 3.150, standard deviation 0.486
Population of 203 members in 36 species
Total extinctions: 0
Generation time: 16.802 sec (15.656 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.56659 stdev: 0.06820
Best fitness: 0.70377 - size: (4, 19) - species 22 - id 811
Average adjusted fitness: 0.211
Mean genetic distance 3.179, standard deviation 0.491
Population of 207 members in 38 species
Total extinctions: 0
Generation time: 17.438 sec (15.953 average)

 *****

In [9]:
len(winner_list)

100

In [10]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-mnist2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [11]:
def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(10)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = digits.images.reshape(digits.images.shape[0], digits.images.shape[1] * digits.images.shape[2])[j]
    test_y = digits.target[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

In [12]:
from random import randint
pred_value = []
error_list = []

for j in range(1617,1617+180):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(10):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == digits.target[1617:])/180

0.8388888888888889

In [13]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(list_P, digits.target[1617:])
print(cm)

[[16  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 17  0  0  0  0  1  2  0]
 [ 0  0  0  8  0  0  0  0  1  2]
 [ 0  0  0  0 18  1  0  2  0  0]
 [ 0  0  0  3  0 17  0  0  0  2]
 [ 0  0  0  1  0  0 18  0  0  0]
 [ 0  0  0  1  1  0  0 16  1  0]
 [ 0  1  0  5  1  0  0  0 11  2]
 [ 0  0  0  0  0  0  0  0  2 12]]


In [14]:
import seaborn as sn


df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.xlabel('Real Label')
plt.ylabel('Prediction Label')
plt.show()

In [15]:
list_P = []
for i,p_value in enumerate(pred_value):
    print("#{} min_error:{} index:{} real:{}".format(i, error_list[i], p_value[0], digits.target[1617+i]))

#0 min_error:27 index:[4 5] real:5
#1 min_error:29 index:[2] real:2
#2 min_error:25 index:[8] real:8
#3 min_error:7 index:[0] real:0
#4 min_error:24 index:[1] real:1
#5 min_error:16 index:[7] real:7
#6 min_error:11 index:[6] real:6
#7 min_error:29 index:[3] real:3
#8 min_error:12 index:[2] real:2
#9 min_error:26 index:[1] real:1
#10 min_error:35 index:[2] real:7
#11 min_error:31 index:[7] real:4
#12 min_error:9 index:[6] real:6
#13 min_error:29 index:[3] real:3
#14 min_error:27 index:[1] real:1
#15 min_error:35 index:[3] real:3
#16 min_error:31 index:[5 7] real:9
#17 min_error:22 index:[1] real:1
#18 min_error:30 index:[7] real:7
#19 min_error:10 index:[6] real:6
#20 min_error:28 index:[8] real:8
#21 min_error:9 index:[4] real:4
#22 min_error:28 index:[3] real:3
#23 min_error:29 index:[1] real:1
#24 min_error:13 index:[4] real:4
#25 min_error:10 index:[0] real:0
#26 min_error:21 index:[5] real:5
#27 min_error:19 index:[3] real:3
#28 min_error:23 index:[6] real:6
#29 min_error:26 index:

In [16]:
pred_value = []
error_list = []

for j in range(1617):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(10):
        error.append(10 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Testset acc:{}".format(np.sum(list_P == digits.target[:1617])/1617))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Testset acc:0.8855905998763142
Avg Base Learner:0.8029870129870129


In [17]:
from random import randint

list_acc = []

for iii in range(necoc):
    pred_value = []
    error_list = []


    for j in range(1617,1617+180):
        pred = []
        for i in range(iii):
            [pred_outputs, real_outputs] = get_pred_real(i, j)
            #print(pred_outputs, real_outputs)
            pred.append(pred_outputs)
        #print(np.array(pred).T)

        error = []
        for i in range(10):
            error.append(iii - np.sum(number_codes[i][:iii] == np.array(pred).T))
        #print(error)

        pred_value.append(np.where(error==np.min(error)))
        error_list.append(np.min(error))
        #print(np.where(error==np.min(error)) )
    list_P = []
    for i in pred_value:
        if (len(i[0])) == 1:
            list_P.append(i[0][0])
        else:
            random_pick = randint(0, len(i[0])-1)
            list_P.append(i[0][random_pick])
    acc = np.sum(list_P == digits.target[1617:])/180
    print("Accuracy on training set: {}".format(acc))
    list_acc.append(acc)

Accuracy on training set: 0.09444444444444444
Accuracy on training set: 0.16666666666666666
Accuracy on training set: 0.21666666666666667
Accuracy on training set: 0.2833333333333333
Accuracy on training set: 0.38333333333333336
Accuracy on training set: 0.48333333333333334
Accuracy on training set: 0.4777777777777778
Accuracy on training set: 0.4888888888888889
Accuracy on training set: 0.45555555555555555
Accuracy on training set: 0.5055555555555555
Accuracy on training set: 0.49444444444444446
Accuracy on training set: 0.5111111111111111
Accuracy on training set: 0.55
Accuracy on training set: 0.5166666666666667
Accuracy on training set: 0.5388888888888889
Accuracy on training set: 0.5444444444444444
Accuracy on training set: 0.5666666666666667
Accuracy on training set: 0.5555555555555556
Accuracy on training set: 0.5666666666666667
Accuracy on training set: 0.5611111111111111
Accuracy on training set: 0.6
Accuracy on training set: 0.6
Accuracy on training set: 0.6444444444444445
Ac

In [18]:
plt.plot(range(necoc),list_acc)
plt.show()

In [19]:
list_acc

[0.09444444444444444,
 0.16666666666666666,
 0.21666666666666667,
 0.2833333333333333,
 0.38333333333333336,
 0.48333333333333334,
 0.4777777777777778,
 0.4888888888888889,
 0.45555555555555555,
 0.5055555555555555,
 0.49444444444444446,
 0.5111111111111111,
 0.55,
 0.5166666666666667,
 0.5388888888888889,
 0.5444444444444444,
 0.5666666666666667,
 0.5555555555555556,
 0.5666666666666667,
 0.5611111111111111,
 0.6,
 0.6,
 0.6444444444444445,
 0.6166666666666667,
 0.6611111111111111,
 0.65,
 0.6611111111111111,
 0.6777777777777778,
 0.7333333333333333,
 0.7111111111111111,
 0.7222222222222222,
 0.7388888888888889,
 0.75,
 0.75,
 0.7722222222222223,
 0.7666666666666667,
 0.7666666666666667,
 0.7833333333333333,
 0.7888888888888889,
 0.7888888888888889,
 0.7944444444444444,
 0.7611111111111111,
 0.7555555555555555,
 0.7944444444444444,
 0.7888888888888889,
 0.7888888888888889,
 0.7944444444444444,
 0.8055555555555556,
 0.7888888888888889,
 0.7944444444444444,
 0.7777777777777778,
 0.78333